<a href="https://colab.research.google.com/github/manas95826/Summarize-Extension/blob/main/Summarize_URL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q openai langchain unstructured tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.2/806.2 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.4/252.4 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 26.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 28.0 MB/

In [2]:
from langchain_community.llms import HuggingFaceEndpoint
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
%pip install --upgrade --quiet huggingface_hub
from langchain.docstore.document import Document
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms.openai import OpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.document_loaders import UnstructuredURLLoader

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.2/346.2 kB 5.9 MB/s eta 0:00:00


In [35]:
urls = ["https://tech.hindustantimes.com/tech/news/samsung-galaxy-s23-ultra-5g-now-available-with-an-enticing-price-cut-on-amazon-71709457508945.html"]

In [36]:
loader = UnstructuredURLLoader(urls=urls)

In [37]:
data = loader.load()

In [38]:
chunk_size = 3000
chunk_overlap = 200

text_splitter = CharacterTextSplitter(
    # separator = "\n\n"
  chunk_size=chunk_size, # Maximum size of a chunk
  chunk_overlap=chunk_overlap, # Maintain continuity, have some overlap of chunks
  length_function=len, # Count number of characters to measure chunk size
)
texts = text_splitter.split_text( data[0].page_content)

# Create Document objects for each text chunk
docs = [Document(page_content=t) for t in texts[:]]

In [39]:
docs

[Document(page_content="Saved Articles\n\nNews\n\nMobile        Mobile News     Mobile Reviews  Mobile Finder Mobile Comparison  Mobile Recommender\n\nLaptops PC         Laptops/PC News       Laptops/PC Reviews  Laptop Finder Laptop Comparison  Laptop Recommender\n\nGaming        Gaming News     Gaming Reviews\n\nTV         TV News       TV Reviews\n\nGadgets   Mobile Finder  Laptop Finder  Tablet Finder\n\nWearables          Wearables News  Wearables Reviews  Smart Watches  Fitness Bands\n\nCompare  Mobile Comparison Laptop Comparison Tablet Comparison\n\nPhotos\n\nVideos\n\nWeb Stories\n\nHow To\n\nMore  Samsung Event   Apple Event For You Home Appliances Brand Stories  Mobile Recommender  Laptop Recommender  Tablet Recommender\n\nAbout Us\n\nContact Us\n\nSitemap\n\nRSS\n\nTerms of Use\n\nPrivacy Policy\n\nCopyright © HT Media Limited\n\nAll rights reserved.\n\nHome\n\nTech\n\nNews    Samsung Galaxy S23 Ultra 5G now available with an enticing price cut on Amazon!\n\nSamsung Galaxy S

In [17]:
from getpass import getpass

HUGGINGFACEHUB_API_TOKEN = getpass()

··········


In [18]:
import os

os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

In [42]:
template = """Create a comprehensive summary covering all aspects for the newspaper. {docs}
Provide a concise headline and main news summary."""

prompt = PromptTemplate.from_template(template)

In [43]:
repo_id = "mistralai/Mistral-7B-Instruct-v0.2"

llm = HuggingFaceEndpoint(
    repo_id=repo_id, max_length=1024, temperature=0.5, token=HUGGINGFACEHUB_API_TOKEN
)
llm_chain = LLMChain(prompt=prompt, llm=llm)
print(llm_chain.run(docs))

                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.
                    token was transferred to model_kwargs.
                    Please make sure that token is what you intended.


Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


Headline: Samsung Galaxy S23 Ultra 5G Receives Significant Price Cut on Amazon

Main Summary: The Samsung Galaxy S23 Ultra 5G, a premium smartphone with impressive features and specifications, has received a substantial price cut on Amazon, making it an attractive option for smartphone enthusiasts. The device is now available for Rs. 1,09,999, down from its original price of Rs. 1,49,999. Customers can also benefit from various offers and discounts, including bank offers, no-cost EMI options, and significant discounts when exchanging their old devices. With its powerful capabilities and attractive price, the Samsung Galaxy S23 Ultra 5G presents an irresistible proposition for those in search of a premium smartphone experience.
